In [1]:
import numpy as np
from skimage.feature import peak_local_max
from skimage import feature
from scipy.signal import argrelextrema
from scipy.signal import find_peaks
from skimage.segmentation import watershed

#The scipy.signal stuff is a red herring, but left here for as a 
#reminder of different possible options 
#
# However skimage has tools to solve all parts with very little 
#coding required.

## Data input/processing

In [2]:
def process_line(l):
    return [int(x) for x in list(l.strip())]

In [3]:
sample = ['2199943210',
'3987894921',
'9856789892',
'8767896789',
'9899965678']


with open("9.txt") as f:
    contents = f.readlines()
    

In [4]:
s = np.array([process_line(l) for l in sample])
c = np.array([process_line(l) for l in contents])
s,c

(array([[2, 1, 9, 9, 9, 4, 3, 2, 1, 0],
        [3, 9, 8, 7, 8, 9, 4, 9, 2, 1],
        [9, 8, 5, 6, 7, 8, 9, 8, 9, 2],
        [8, 7, 6, 7, 8, 9, 6, 7, 8, 9],
        [9, 8, 9, 9, 9, 6, 5, 6, 7, 8]]),
 array([[9, 8, 7, ..., 3, 4, 7],
        [6, 9, 8, ..., 4, 5, 6],
        [5, 5, 9, ..., 6, 8, 9],
        ...,
        [8, 9, 6, ..., 7, 8, 9],
        [7, 6, 4, ..., 8, 9, 3],
        [8, 4, 3, ..., 9, 1, 2]]))

## Part One

First a check that the numpy pad does what we need it to do before tossing the array into peak_local_max to find all of the points we need to score.

In [5]:
sp = np.pad(s, ((1, 1), (1, 1)), 'maximum')
sp

array([[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9],
       [9, 2, 1, 9, 9, 9, 4, 3, 2, 1, 0, 9],
       [9, 3, 9, 8, 7, 8, 9, 4, 9, 2, 1, 9],
       [9, 9, 8, 5, 6, 7, 8, 9, 8, 9, 2, 9],
       [9, 8, 7, 6, 7, 8, 9, 6, 7, 8, 9, 9],
       [9, 9, 8, 9, 9, 9, 6, 5, 6, 7, 8, 9],
       [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]])

In [6]:
def find_minimums(sarray):
    spad = np.pad(sarray, ((1, 1), (1, 1)), 'maximum')
    result = peak_local_max(-spad)-1
    return result

In [7]:
find_minimums(s)

array([[0, 9],
       [0, 1],
       [2, 2],
       [4, 6]])

In [8]:
find_minimums(c)

array([[ 0, 15],
       [63, 91],
       [31, 41],
       [30, 81],
       [65, 26],
       [66,  9],
       [29, 73],
       [29, 53],
       [66, 96],
       [28, 17],
       [68, 69],
       [27, 25],
       [63, 13],
       [69, 50],
       [26, 34],
       [25, 67],
       [25, 63],
       [70, 80],
       [71, 23],
       [51, 20],
       [75, 64],
       [22, 55],
       [76, 37],
       [76, 47],
       [76, 71],
       [26, 79],
       [62, 56],
       [35,  4],
       [35, 36],
       [51, 10],
       [50, 27],
       [53, 77],
       [47, 80],
       [54, 33],
       [45, 60],
       [55,  1],
       [45, 29],
       [55, 26],
       [56, 14],
       [57, 89],
       [43,  4],
       [42, 93],
       [42, 46],
       [40, 84],
       [61, 62],
       [39, 55],
       [38, 99],
       [38, 25],
       [38, 17],
       [37, 47],
       [36, 77],
       [36, 70],
       [36, 63],
       [35, 91],
       [76, 93],
       [78, 26],
       [75, 53],
       [88, 46],
       [ 9, 46

In [9]:
def part_one_score(data):
    mvals = find_minimums(data)
    score = 0
    for a,b in mvals:
        score = score + data[a,b] + 1
    return score

In [10]:
part_one_score(s)

15

In [11]:
part_one_score(c)

541

## Part Two

Here we need to do watershed.  Which if you apply skimage's watershed implementation out of the box, it is almost a complete solution.  The only minor change is that you need to remove the 9's from the regions.  Hence there are no functions in this part because a few cells of commands produced an answer.

In [12]:
ws = watershed(s)
ws

array([[1, 1, 1, 3, 2, 2, 2, 2, 2, 2],
       [1, 1, 3, 3, 3, 2, 2, 2, 2, 2],
       [1, 3, 3, 3, 3, 3, 2, 4, 2, 2],
       [3, 3, 3, 3, 3, 4, 4, 4, 4, 2],
       [3, 3, 3, 3, 4, 4, 4, 4, 4, 4]], dtype=int32)

In [13]:
ws[s == 9] = 0
ws

array([[1, 1, 0, 0, 0, 2, 2, 2, 2, 2],
       [1, 0, 3, 3, 3, 0, 2, 0, 2, 2],
       [0, 3, 3, 3, 3, 3, 0, 4, 0, 2],
       [3, 3, 3, 3, 3, 0, 4, 4, 4, 0],
       [0, 3, 0, 0, 0, 4, 4, 4, 4, 4]], dtype=int32)

In [14]:
wsc = watershed(c)
wsc

array([[ 20,   8,   8, ...,  12,  12,  12],
       [ 20,  20,   8, ...,  12,  12,  12],
       [ 20,  20,  20, ...,  12,  12,  12],
       ...,
       [238, 238, 238, ..., 227, 227, 245],
       [238, 238, 238, ..., 227, 245, 245],
       [238, 238, 238, ..., 245, 245, 245]], dtype=int32)

In [15]:
wsc[c == 9] = 0
wsc

array([[  0,   8,   8, ...,  12,  12,  12],
       [ 20,   0,   8, ...,  12,  12,  12],
       [ 20,  20,   0, ...,  12,  12,   0],
       ...,
       [238,   0, 238, ..., 227, 227,   0],
       [238, 238, 238, ..., 227,   0, 245],
       [238, 238, 238, ...,   0, 245, 245]], dtype=int32)

In [16]:
labels,counts = np.unique(wsc,return_counts=True)

In [17]:
labels

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [18]:
counts.sort()

In [19]:
counts

array([   2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    3,    3,    3,    3,    3,    3,    3,    3,    3,
          3,    3,    3,    3,    3,    3,    3,    4,    4,    4,    4,
          4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
          5,    5,    5,    5,    5,    5,    5,    6,    6,    6,    6,
          6,    6,    6,    6,    6,    7,    7,    7,    7,    8,    8,
          8,    8,    9,    9,    9,    9,    9,    9,    9,    9,   10,
         10,   11,   11,   11,   11,   11,   12,   12,   12,   13,   13,
         13,   14,   14,   14,   14,   14,   15,   15,   15,   15,   15,
         15,   16,   16,   16,   16,   17,   17,   17,   17,   19,   20,
         20,   20,   20,   21,   21,   22,   22,   23,   23,   23,   24,
         24,   24,   25,   26,   26,   26,   26,   26,   27,   27,   28,
         28,   28,   28,   29,   29,   30,   30,   30,   30,   31,   31,
         31,   32,   32,   32,   33,   33,   33,   

In [20]:
92*94*98

847504